### EEG-based classification of imagined digits using a recurrent neural network  


In [1]:
import numpy as np
import numpy.fft as fft
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import signal
import sklearn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
import torchaudio
from torchvision import transforms
import cv2
import time, sys, os

from utils import MindBigData, GetDataSet, GetDataLoaders, GetDataLoadersEEGImages, GetDataAndPreProcess
from MultilayerBidirectionalLSTM import MultilayerBidirectionalLSTM

seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%matplotlib inline


C:\Users\ryoii\AppData\Roaming\Python\Python310\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


### Signal Acquisition & Preprocessing

In [2]:
"""
Load EEG data from specified data path, with specified number of samples per digit.
- GetDataSet : standardization, min-max scaling within std 
    x : 振幅 in microVolts
    y : target digit
- Preprocess : butterworth filter(highpass 0.1Hz + 平滑化),
 notch filter(ノイズ除去 50Hz), trim first 32 samples
"""
dataName = "EP/EP1.01.txt"
# dataName = "MUSE/MU.txt"
dataPath = os.path.join(os.path.dirname(os.getcwd()), dataName)

x_raw, x_preprocessed, x_reconstructed, x_standardized, y = GetDataAndPreProcess(input_file=dataPath, samples_per_digit=10000)

[6482, 6324, 6450, 6582, 6309, 6527, 6486, 6296, 6497, 6517]
PreProcess - complete
Wavelet transformation - complete
Standardization - complete


In [3]:
# (trial, channel, data points)
print(x_raw.shape)
print(x_preprocessed.shape)
print(x_reconstructed.shape)
print(x_standardized.shape)
print(y.shape)


(64470, 14, 256)
(64470, 14, 256)
(64470, 14, 256)
(64470, 14, 256)
(64470,)


In [4]:
"""
Prepare dataloaders 75%:15%:10%
"""
train_loader_raw, valid_loader_raw, test_loader_raw = GetDataLoaders(x_raw, y, batch_size=1024)
train_loader_preprocessed, valid_loader_preprocessed, test_loader_preprocessed = GetDataLoaders(x_preprocessed, y, batch_size=1024)
train_loader_reconstructed, valid_loader_reconstructed, test_loader_reconstructed = GetDataLoaders(x_reconstructed, y, batch_size=1024)
train_loader_standardized, valid_loader_standardized, test_loader_standardized = GetDataLoaders(x_standardized, y, batch_size=1024)

train_iter, validation_iter, test_iter = iter(train_loader_standardized), iter(valid_loader_standardized), iter(test_loader_standardized)

# Check the shapes of inputs and targets
print("train - Input shape:", next(train_iter)[0].shape)
print("train - Target shape:", next(train_iter)[1].shape)
print("validation - Input shape:", next(validation_iter)[0].shape)
print("validation - Target shape:", next(validation_iter)[1].shape)
print("test - Input shape:", next(test_iter)[0].shape)
print("test - Target shape:", next(test_iter)[1].shape)


train - Input shape: torch.Size([1024, 14, 256])
train - Target shape: torch.Size([1024])
validation - Input shape: torch.Size([1024, 14, 256])
validation - Target shape: torch.Size([1024])
test - Input shape: torch.Size([1024, 14, 256])
test - Target shape: torch.Size([1024])


### Deep Learning Classifier

In [5]:
# Initialize the model
model = MultilayerBidirectionalLSTM(input_shape=(14, 256), num_classes=10)

# Train the model and save the best one
model.train(train_loader=train_loader_standardized, valid_loader=valid_loader_standardized, epochs=100, checkpoint_path='best_model.keras')

# Evaluate the best model on the test set
test_loss, test_acc = model.evaluate(test_loader_standardized)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

Epoch 1/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1004 - loss: 2.3028
Epoch 1: val_loss improved from inf to 2.30276, saving model to best_model.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.1004 - loss: 2.3029 - val_accuracy: 0.0953 - val_loss: 2.3028
Epoch 2/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1037 - loss: 2.3027
Epoch 2: val_loss improved from 2.30276 to 2.30264, saving model to best_model.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 94s 2s/step - accuracy: 0.1037 - loss: 2.3027 - val_accuracy: 0.1065 - val_loss: 2.3026
Epoch 3/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.0995 - loss: 2.3026
Epoch 3: val_loss did not improve from 2.30264
48/48 ━━━━━━━━━━━━━━━━━━━━ 101s 2s/step - accuracy: 0.0995 - loss: 2.3026 - val_accuracy: 0.1065 - val_loss: 2.3031
Epoch 4/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1002 - loss: 2.3027
Epoch 4: val_loss did not improve from 2.30264
48/48 ━━━━━━━━━━━━━━━━━━━━ 98s 2s/step - accuracy: 0.1002 

In [ ]:
# # Display 2 signals with their labels
# data_iter = iter(train_loader_raw)
# inputs, labels = next(data_iter)
# EEGChannel = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]

# fig = plt.figure(figsize=(20, 15))
# for i in range(1):
#     for j in range(14):
#         ax = fig.add_subplot(14, 2, i + 1 + 2 * j)
#         if (j == 0):
#             ax.set_title(
#                 'Raw : digit seen by patient: ' + str(labels[i].item()))  # .item() gets the value contained in a tensor
#         ax.plot(inputs[i][j])
#         ax.set_xlabel('samples')
#         ax.set_ylabel(EEGChannel[j])
#         ax.grid()

In [ ]:
# # Display 2 signals with their labels
# data_iter = iter(train_loader_preprocessed)
# inputs, labels = next(data_iter)
# EEGChannel = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]

# fig = plt.figure(figsize=(20, 15))
# for i in range(1):
#     for j in range(14):
#         ax = fig.add_subplot(14, 2, i + 1 + 2 * j)
#         if (j == 0):
#             ax.set_title(
#                 'Preprocessed : digit seen by patient: ' + str(labels[i].item()))  # .item() gets the value contained in a tensor
#         ax.plot(inputs[i][j])
#         ax.set_xlabel('samples')
#         ax.set_ylabel(EEGChannel[j])
#         ax.grid()

In [ ]:
# # Display 2 signals with their labels
# data_iter = iter(train_loader_reconstructed)
# inputs, labels = next(data_iter)
# EEGChannel = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]

# fig = plt.figure(figsize=(20, 15))
# for i in range(1):
#     for j in range(14):
#         ax = fig.add_subplot(14, 2, i + 1 + 2 * j)
#         if (j == 0):
#             ax.set_title(
#                 'reconstructed : digit seen by patient: ' + str(labels[i].item()))  # .item() gets the value contained in a tensor
#         ax.plot(inputs[i][j])
#         ax.set_xlabel('samples')
#         ax.set_ylabel(EEGChannel[j])
#         ax.grid()

In [ ]:
# # Display 2 signals with their labels
# data_iter = iter(train_loader_standardized)
# inputs, labels = next(data_iter)
# EEGChannel = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]

# fig = plt.figure(figsize=(20, 15))
# for i in range(1):
#     for j in range(14):
#         ax = fig.add_subplot(14, 2, i + 1 + 2 * j)
#         if (j == 0):
#             ax.set_title(
#                 'standardized : digit seen by patient: ' + str(labels[i].item()))  # .item() gets the value contained in a tensor
#         ax.plot(inputs[i][j])
#         ax.set_xlabel('samples')
#         ax.set_ylabel(EEGChannel[j])
#         ax.grid()